In [14]:
import mne
from pathlib import Path
import numpy as np
from mnelab.io.writers import write_edf

ImportError: 
    Importing PySide6 disabled by IPython, which has
    already imported an Incompatible QT Binding: pyqt5
    

In [12]:
# same function as mne but support nan values (const physical calues)
def write_edf_nan(fname, raw):
    """Export raw to EDF/BDF file (requires pyEDFlib)."""
    import pyedflib

    suffixes = Path(fname).suffixes
    ext = "".join(suffixes[-1:])
    if ext == ".edf":
        filetype = pyedflib.FILETYPE_EDFPLUS
        dmin, dmax = -32768, 32767
    elif ext == ".bdf":
        filetype = pyedflib.FILETYPE_BDFPLUS
        dmin, dmax = -8388608, 8388607
    data = raw.get_data() * 1e6  # convert to microvolts
    fs = raw.info["sfreq"]
    nchan = raw.info["nchan"]
    ch_names = raw.info["ch_names"]
    if raw.info["meas_date"] is not None:
        meas_date = raw.info["meas_date"]
    else:
        meas_date = None
    prefilter = (f"{raw.info['highpass']}Hz - "
                 f"{raw.info['lowpass']}")
    pmin, pmax = data.min(axis=1), data.max(axis=1)
    f = pyedflib.EdfWriter(fname, nchan, filetype)
    channel_info = []
    data_list = []
    for i in range(nchan):
        channel_info.append(dict(label=ch_names[i],
                                 dimension="uV",
                                 sample_rate=fs,
                                 physical_min=-5000,
                                 physical_max=5000,
                                 digital_min=dmin,
                                 digital_max=dmax,
                                 transducer="",
                                 prefilter=prefilter))
        data_list.append(data[i])
    f.setTechnician("Exported by MNELAB")
    f.setSignalHeaders(channel_info)
    if raw.info["meas_date"] is not None:
        f.setStartdatetime(meas_date)
    # note that currently, only blocks of whole seconds can be written
    f.writeSamples(data_list)
    for annot in raw.annotations:
        f.writeAnnotation(annot["onset"], annot["duration"], annot["description"])

In [3]:
import mne
subj = '53'
depth = ['RAH1', 'LAH1', 'RA1', 'LA1', 'LEC1', 'REC1', 'RPHG1', 'LPHG1', 'RMH1', 'LMH1', 'RAH2', 'LAH2', 'RA2', 'LA2', 'LEC2', 'REC2', 'RPHG2', 'LPHG2', 'RMH2', 'LMH2']
if subj in ['018', '025']:
    depth = [x for x in depth+['RA3', 'RA4'] if x not in ['RA1', 'RA2']]
raw = mne.io.read_raw_edf(f'D:\\TLV\\P{subj}_full_filtered_mtl.edf')
# raw = mne.io.read_raw_edf(f'D:\\TLV\\P{subj}_full_filtered_mtl.edf')
raw.set_channel_types({x: 'eog' for x in ['EOG1', 'EOG2']})
raw.pick_channels(depth + ['EOG1', 'EOG2'])
raw.set_channel_types({x: 'seeg' for x in depth if x in raw.ch_names})
# # copy annotations from other file
# raw_an = mne.io.read_raw(fr"D:\UCLA\{subj}_clean_mtl_annot.fif")
# annot = raw_an.annotations
# raw = raw.set_annotations(mne.Annotations(annot.onset, annot.duration, annot.description))
raw.plot(duration=60 * 5, scalings=dict(eeg=5e-4, eog=4e-4, seeg=5e-4))

Extracting EDF parameters from D:\TLV\P53_full_filtered_mtl.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\user\AppData\Local\Temp\ipykernel_27300\1735629148.py:9: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in an error because the following channel names are missing:
['LAH1', 'LA1', 'LEC1', 'REC1', 'RPHG1', 'LPHG1', 'RMH1', 'LMH1', 'LAH2', 'LA2', 'LEC2', 'REC2', 'RPHG2', 'LPHG2', 'RMH2', 'LMH2']
Either fix your included names or explicitly pass ordered=False.
  raw.pick_channels(depth + ['EOG1', 'EOG2'])
C:\Users\user\AppData\Local\Temp\ipykernel_27300\1735629148.py:9: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(depth + ['EOG1', 'EOG2'])


In [3]:
# save the fif with annotations
raw.drop_channels(raw.info['bads'], on_missing='ignore')
if subj == '025':
    # copy eog1 as 2
    raw.load_data()
    # eog2 = mne.io.RawArray(raw.copy().pick_channels(['EOG1']).get_data(),
    #                        mne.create_info(ch_names=['EOG2'], sfreq=raw.info['sfreq']))
    # raw.add_channels([eog2])
if subj == '48':
    raw.crop(tmax=60*392)
if subj == '51':
    raw.crop(tmax=60*530)
if subj == '404':
    raw.crop(tmin=93*60, tmax=60*617)
if subj == '405':
    raw.crop(tmin=909*60)
if subj == '422':
    raw.crop(tmin=57*60)
raw.save(f'D:\\TLV\\{subj}_clean_mtl_annot.fif', overwrite=True)
# save an edf with nans
output_filename = 'clean_mtl'
nan_clean = raw.get_data(reject_by_annotation='NaN')
clean_raw = mne.io.RawArray(nan_clean, raw.info)

Writing D:\TLV\38_clean_mtl_annot.fif


C:\Users\user\AppData\Local\Temp\ipykernel_3772\2202309907.py:19: RuntimeWarning: This filename (D:\TLV\38_clean_mtl_annot.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(f'D:\\TLV\\{subj}_clean_mtl_annot.fif', overwrite=True)


Closing D:\TLV\38_clean_mtl_annot.fif
Writing D:\TLV\38_clean_mtl_annot-1.fif
Closing D:\TLV\38_clean_mtl_annot-1.fif
[done]
Setting 1787181 of 37045000 (4.82%) samples to NaN, retaining 35257819 (95.18%) samples.
Creating RawArray with float64 data, n_channels=27, n_times=37045000
    Range : 0 ... 37044999 =      0.000 ... 37044.999 secs
Ready.


In [11]:
write_edf_nan(f'D:\\TLV\\{subj}_clean_mtl.edf', clean_raw)

C:\Users\user\anaconda3\envs\depth_ieds\Lib\site-packages\pyedflib\edfwriter.py:31: UserWarning: Invalid char: header entries should contain only ASCII characters and no spaces: "Exported by MNELAB"
  warnings.warn('Invalid char: header entries should contain only ASCII'\


In [7]:
subj = '018'
x = mne.io.read_raw(f'C:\\UCLA\\{subj}_clean_mtl_annot.fif')
nan_clean = x.get_data(reject_by_annotation='NaN')
clean_raw = mne.io.RawArray(nan_clean, x.info)
write_edf_nan(f'C:\\UCLA\\{subj}_clean_mtl.edf', clean_raw)

Opening raw data file C:\UCLA\018_clean_mtl_annot.fif...


C:\Users\user\AppData\Local\Temp/ipykernel_8100/2360845214.py:2: RuntimeWarning: This filename (C:\UCLA\018_clean_mtl_annot.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  x = mne.io.read_raw(f'C:\\UCLA\\{subj}_clean_mtl_annot.fif')


Isotrak not found
    Range : 0 ... 5344999 =      0.000 ...  5344.999 secs
Ready.
Setting 1582808 of 5345000 (29.61%) samples to NaN, retaining 3762192 (70.39%) samples.
Creating RawArray with float64 data, n_channels=6, n_times=5345000
    Range : 0 ... 5344999 =      0.000 ...  5344.999 secs
Ready.


In [5]:
raw.plot(duration=30, scalings=dict(eeg=5e-4, eog=4e-4, seeg=5e-4))

In [15]:
# 49 don't have mtl file but full filtered is ok
subj = '53'
depth = ['RAH1', 'LAH1', 'RA1', 'LA1', 'LEC1', 'REC1', 'RPHG1', 'LPHG1', 'RMH1', 'LMH1',
         'RAH2', 'LAH2', 'RA2', 'LA2', 'LEC2', 'REC2', 'RPHG2', 'LPHG2', 'RMH2', 'LMH2']
scalp_all = ['C3', 'C4', 'PZ', 'EOG1', 'EOG2', 'F4', 'P4', 'F10', 'T10', 'F3', 'P3', 'F9', 'T9', 'CZ', 'P4', 'F8', 'T8', 'P8', 'O1', 'O2', 'T5', 'C6', 'P6', 'F7', 'C5', 'P5', 'FZ']
raw = mne.io.read_raw_edf(f'D:\\TLV\\P{subj}_full_filtered_mtl.edf')
curr_scalp = [x for x in scalp_all if x in raw.ch_names]
curr_depth = [x for x in depth if x in raw.ch_names]
raw.pick_channels(curr_scalp + curr_depth)
raw.set_channel_types({x: 'eog' for x in ['EOG1', 'EOG2']})
raw.set_channel_types({x: 'seeg' for x in depth if x in raw.ch_names})
raw.plot(duration=60 * 5, scalings=dict(eeg=5e-5, eog=4e-5, seeg=5e-4))

Extracting EDF parameters from D:\TLV\P53_full_filtered_mtl.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\user\AppData\Local\Temp\ipykernel_27300\3391153837.py:9: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(curr_scalp + curr_depth)


In [7]:
raw.info['bads']

['P4']

In [8]:
# save the fif with annotations
raw.drop_channels(raw.info['bads'], on_missing='ignore')
raw.save(f'D:\\TLV\\{subj}_clean_mtl_annot.fif', overwrite=True)
# save an edf with nans
output_filename = 'clean_mtl'
nan_clean = raw.get_data(reject_by_annotation='NaN')
clean_raw = mne.io.RawArray(nan_clean, raw.info)

Writing D:\TLV\53_clean_mtl_annot.fif


C:\Users\user\AppData\Local\Temp\ipykernel_27300\1249924845.py:3: RuntimeWarning: This filename (D:\TLV\53_clean_mtl_annot.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(f'D:\\TLV\\{subj}_clean_mtl_annot.fif', overwrite=True)


Closing D:\TLV\53_clean_mtl_annot.fif
[done]
Setting 4582759 of 26127000 (17.54%) samples to NaN, retaining 21544241 (82.46%) samples.
Creating RawArray with float64 data, n_channels=13, n_times=26127000
    Range : 0 ... 26126999 =      0.000 ... 26126.999 secs
Ready.


NameError: name 'write_edf_nan' is not defined

In [13]:
write_edf_nan(f'D:\\TLV\\{subj}_clean_mtl.edf', clean_raw)


C:\Users\user\anaconda3\envs\depth_ieds\Lib\site-packages\pyedflib\edfwriter.py:31: UserWarning: Invalid char: header entries should contain only ASCII characters and no spaces: "Exported by MNELAB"
  warnings.warn('Invalid char: header entries should contain only ASCII'\
